<a href="https://colab.research.google.com/github/JhonnyLimachi/Sigmoidal/blob/main/52_Limpeza_de_dados%2C_outliers_e_valores_ausentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center"><img src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/eds.png" height="100px"></p>

# Limpeza de dados, outliers e valores ausentes

Neste projeto, vamos focar em uma habilidade básica, mas essencial de um Cientista de Dados. Saber lidar com valores ausentes, lidar com *outliers*, transformar os dados. Essas são algumas das atividades que fazem parte do checklist de todos os projetos que executamos.


Esses processos podem ter um impacto enorme nos resultados, desde a parte da análise dos dados até os modelos de *Machine Learning*.
<p align="center"><img src="https://image.freepik.com/free-vector/group-analysts-working-graphs_1262-21249.jpg
"></p>


Aqui, veremos o que buscar entender nos dados, os passos a dar, como identificar, e tratar *outliers*, as melhores práticas e métodos para lidar com dados ausentes, e por fim, converter os dados para os formatos necessários.


## O Que Analisar?

Primeiro de tudo, precisamos entender o que são bons dados, para que saibamos os passos que precisamos tomar para ter o melhor conjunto de dados possível, a partir dos dados que estamos utilizando.

* Tipo de Dados
  * Coluna de data em formato `datetime`, coluna de valores monetários em `float`.
* *Range* dos Dados
  * Meses de 1 a 12, dias do mês de 1 a 31.
* Dados Obrigatórios
  * Algumas colunas não podem estar vazias
* Dados Únicos
  * CPF, RG, CNPJ, ID de usuário.
* Dados Categóricos
  * Gênero Masculino ou Feminino.
* Padrões regulares
  * (61) 9 8765-4321
* Validade Entre os Campos
  * Data de saída não ser anterior à data de entrada.


In [1]:
# importando os pacotes necessários
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# configurando a visualização
sns.set_style()
%matplotlib inline

## Os Dados

Utilizaremos mais de um conjunto de dados nesse projeto, mas o primeiro deles será do mesmo grupo usado no Projeto do Módulo 2 do curso Data Science na Prática.

Nesse projeto, analisamos dados sobre a violência no Rio de Janeiro.

<p align="center"><img src="https://image.freepik.com/free-vector/brazilian-carnival-concept-with-dancing-people-nature_1284-27444.jpg
", width="50%"></p>


Esse conjunto de dados é interessante pois está organizado em números de crimes por mês, desde janeiro de 1991, mas alguns dos crimes só começaram a ser registrados algum tempo depois, como veremos abaixo.

In [2]:
# importando os dados
df = pd.read_csv('https://raw.githubusercontent.com/carlosfab/curso_data_science_na_pratica/master/modulo_02/violencia_rio.csv', sep=',')

# verificando as dimensões
print('Dimensões do Dataset',
    '\nVariáveis: ',df.shape[1], "\n"
      'Entradas: ', df.shape[0])

# verificando as primeiras entradas
df.head()


Dimensões do Dataset 
Variáveis:  56 
Entradas:  344


,vano,mes,hom_doloso,lesao_corp_morte,latrocinio,hom_por_interv_policial,tentat_hom,lesao_corp_dolosa,estupro,hom_culposo,...,pessoas_desaparecidas,encontro_cadaver,encontro_ossada,pol_militares_mortos_serv,pol_civis_mortos_serv,indicador_letalidade,indicador_roubo_rua,indicador_roubo_veic,registro_ocorrencias,fase
0,1991,1,657,NaN,15,NaN,162,3051,NaN,NaN,...,NaN,217,NaN,NaN,NaN,672,1348,1174,NaN,3
1,1991,2,732,NaN,17,NaN,175,3421,NaN,NaN,...,NaN,209,NaN,NaN,NaN,749,1395,1097,NaN,3
2,1991,3,713,NaN,25,NaN,216,3613,NaN,NaN,...,NaN,188,NaN,NaN,NaN,738,1385,1265,NaN,3
3,1991,4,634,NaN,20,NaN,200,3211,NaN,NaN,...,NaN,140,NaN,NaN,NaN,654,1540,1415,NaN,3
4,1991,5,650,NaN,20,NaN,146,3051,NaN,NaN,...,NaN,78,NaN,NaN,NaN,670,1266,1449,NaN,3


In [3]:
# verificando o final do dataset
df.tail()

,vano,mes,hom_doloso,lesao_corp_morte,latrocinio,hom_por_interv_policial,tentat_hom,lesao_corp_dolosa,estupro,hom_culposo,...,pessoas_desaparecidas,encontro_cadaver,encontro_ossada,pol_militares_mortos_serv,pol_civis_mortos_serv,indicador_letalidade,indicador_roubo_rua,indicador_roubo_veic,registro_ocorrencias,fase
339,2019,4,360,1.0,11,124.0,466,5573,483.0,172.0,...,408.0,22,3.0,1.0,0.0,496,11040,3755,67797.0,3
340,2019,5,345,2.0,15,172.0,478,4958,465.0,145.0,...,390.0,20,1.0,0.0,0.0,534,11384,3649,68336.0,3
341,2019,6,332,3.0,8,153.0,436,4769,414.0,152.0,...,403.0,20,1.0,3.0,0.0,496,9551,3115,61202.0,3
342,2019,7,309,5.0,10,194.0,399,4740,402.0,140.0,...,400.0,32,7.0,0.0,0.0,518,10071,3198,65817.0,2
343,2019,8,318,1.0,6,170.0,457,4760,460.0,156.0,...,367.0,27,8.0,2.0,0.0,495,9912,3181,65285.0,2


In [4]:
# modificando o tipo
df.lesao_corp_morte[343].astype('int64')

1

In [5]:
# pandas usa floats quando tem dados ausentes
df.dtypes

vano                            int64
mes                             int64
hom_doloso                      int64
lesao_corp_morte              float64
latrocinio                      int64
hom_por_interv_policial       float64
tentat_hom                      int64
lesao_corp_dolosa               int64
estupro                       float64
hom_culposo                   float64
lesao_corp_culposa            float64
roubo_comercio                  int64
roubo_residencia                int64
roubo_veiculo                   int64
roubo_carga                     int64
roubo_transeunte                int64
roubo_em_coletivo               int64
roubo_banco                     int64
roubo_cx_eletronico           float64
roubo_celular                 float64
roubo_conducao_saque          float64
roubo_apos_saque              float64
roubo_bicicleta               float64
outros_roubos                   int64
total_roubos                    int64
furto_veiculos                  int64
furto_transe

## Como lidar com Dados Ausentes

A resposta é: Depende!

Que tipo de dado está ausente? Em qual proporção? De forma aleatória? Todos esses são aspectos que precisamos levar em consideração ao tratar dados ausentes.

Tomando os dados do RJ como exemplo, vemos que temos algumas colunas com quase todos os dados ausentes, mas qual o motivo disso? De onde esses dados são extraídos e qual o processo de coleta deles? Dados ausentes implicam algum significado?

Nesse caso, nossa teoria mais predominante é que os dados não começaram a ser registrados até uma determinada data, e após isso, a coleta foi feita de forma efetiva.

In [6]:
# mostrando a quantidade de dados ausentes por variável
(df.isnull().sum()).sort_values(ascending=False)

furto_bicicleta               276
roubo_bicicleta               276
cmba                          180
posse_drogas                  180
trafico_drogas                180
apreensao_drogas_sem_autor    180
apf                           180
aaapai                        180
cmp                           180
furto_celular                 144
sequestro_relampago           144
extorsao                      144
furto_coletivo                144
roubo_apos_saque              144
roubo_conducao_saque          144
roubo_cx_eletronico           144
estelionato                   144
pol_militares_mortos_serv     144
pol_civis_mortos_serv         144
estupro                       144
pessoas_desaparecidas         132
encontro_ossada               132
roubo_celular                 108
registro_ocorrencias           96
lesao_corp_morte               96
sequestro                      96
ameaca                         96
lesao_corp_culposa             96
hom_culposo                    84
hom_por_interv

In [8]:
# dados ausentes por ano
df.set_index('vano').isna().sum(level=0)

TypeError: sum() got an unexpected keyword argument 'level'

In [9]:
# visualizando
df[273:300]

,vano,mes,hom_doloso,lesao_corp_morte,latrocinio,hom_por_interv_policial,tentat_hom,lesao_corp_dolosa,estupro,hom_culposo,...,pessoas_desaparecidas,encontro_cadaver,encontro_ossada,pol_militares_mortos_serv,pol_civis_mortos_serv,indicador_letalidade,indicador_roubo_rua,indicador_roubo_veic,registro_ocorrencias,fase
273,2013,10,377,3.0,12,38.0,441,7508,520.0,184.0,...,497.0,41,5.0,2.0,1.0,430,6639,2639,68030.0,3
274,2013,11,414,2.0,13,31.0,420,7278,467.0,189.0,...,443.0,45,4.0,2.0,0.0,460,6370,2644,63796.0,3
275,2013,12,461,5.0,17,43.0,532,8207,408.0,195.0,...,483.0,41,6.0,0.0,0.0,526,6626,2893,65609.0,3
276,2014,1,464,3.0,15,50.0,557,9050,555.0,194.0,...,628.0,66,3.0,0.0,0.0,532,7760,3206,75403.0,3
277,2014,2,482,5.0,16,56.0,543,8093,555.0,166.0,...,520.0,63,3.0,1.0,0.0,559,7637,3025,69434.0,3
278,2014,3,510,5.0,10,46.0,544,8357,479.0,214.0,...,563.0,59,2.0,5.0,0.0,571,8208,2947,72915.0,3
279,2014,4,449,1.0,17,37.0,559,7484,469.0,194.0,...,532.0,41,3.0,1.0,0.0,504,7988,3073,67358.0,3
280,2014,5,444,3.0,8,53.0,505,6249,442.0,161.0,...,451.0,32,3.0,2.0,0.0,508,8177,3073,66175.0,3
281,2014,6,377,1.0,18,45.0,503,7229,429.0,185.0,...,447.0,25,3.0,2.0,0.0,441,7897,2635,65397.0,3
282,2014,7,370,4.0,14,57.0,439,6070,442.0,156.0,...,429.0,38,2.0,2.0,0.0,445,8435,2479,67615.0,3


## Tratando os Dados

No caso desse conjunto de dados específico, o ideal é analisar os dados apenas do período em que se tem dados. Especialmente em algumas variáveis onde o volume é muito grande, qualquer tipo de preenchimento poderia enviesar os dados de forma que a análise deixasse de ser relevante.

Para outros casos, podemos considerar as seguintes hipóteses:

* Excluir
  * Se os dados ausentes estão em pequeno número,ocorrem aleatoriamente, e a ausência não carrega significado, é melhor excluir a linha. No caso da coluna, se ainda for possível analisar alguma parte dela, use-a, como é o caso aqui. Mas para algumas situações, o ideal é excluir a coluna.

* Preencher
  * Preencher as entradas com dados ausentes com valores estatísticos como a média, mediana, moda ou zeros.
  * A média é mais útil quando a distribuição dos dados é normal. Em dados com distribuição mais enviesada (*skewed*), a mediana é uma solução mais robusta, pois ela é menos sensível a outliers.
  * Uma `Regressão Linear` também pode ser útil, apesar de sensível a outliers, podem nos ajudar a inserir valores que nos ajudem.
  * Indetificar a entrada ausente com algum valor que indique isso pode ser mais informativo, quando a ausência representa valor. Por exemplo, em dados numéricos preencher com zero, e em categóricos criar uma categoria "Desconhecido". Atenção, pois os zeros não podem ser levados em consideração em análises estatísticas.